## Using R's Nblast in Python

This notebook assumes that you have successfully installed and tested all required packages (`nat` and `elmr`) in R (i.e. nblast works in R)

First, load and initialize required modules

In [2]:
import pymaid
from pymaid import rmaid

print('Tested with pymaid version {0}'.format(pymaid.__version__))

remote_instance = pymaid.CatmaidInstance( 'url', 'http_user', 'http_pw', 'token' )

INFO  : Global CATMAID instance set. (pymaid.fetch)


Tested with pymaid version 0.72.3


You can load neurons, edit them (e.g. prune) and pass them to the nblast wrapper directly. In this example, we will keep it simple and just blast a unedited neuron: an olfactory projection neuron with skeleton ID `16`

See `help(rmaid.nblast)` for details.

In [3]:
nbl_res = rmaid.nblast( 16, remote_instance = remote_instance )

INFO  : DPS database not explicitly provided. Loading local FlyCircuit DB from dpscanon.rds (pymaid.rmaid)
Fetching neurons: 100%|██████████| 1/1 [00:04<00:00,  1.01s/it]
INFO  : Blasting neuron... (pymaid.rmaid)
INFO  : Blasting done in 130 seconds (pymaid.rmaid)


`nbl_res` is instance of the `rmaid.nbl_results` class. It holds all data from the nblasting and offers a (growing) number functions to do stuff with the results. See the help:

In [4]:
help(nbl_res)

Help on nbl_results in module pymaid.rmaid object:

class nbl_results(builtins.object)
 |  Class that holds nblast results and contains wrappers that allow easy
 |  plotting.    
 |  
 |  Attributes
 |  ----------
 |  results :   pandas.Dataframe 
 |              Contains top N results
 |  sc :        Robject
 |              Contains original RNblast forward scores 
 |  scr :       Robject     
 |              Original R Nblast reverse scores (Top N only)
 |  neuron :    R ``catmaidneuron``
 |              The neuron that was nblasted transformed into reference space
 |  xdp :       robject
 |              Dotproduct of the transformed neuron
 |  param :     dict 
 |              Contains parameters used for nblasting
 |  db :        file robject 
 |              Dotproduct database as R object "neuronlistfh"
 |  date :      datetime object
 |              Time of nblasting    
 |  
 |  Examples
 |  --------
 |  >>> import pymaid
 |  >>> # Blast neuron by skeleton ID
 |  >>> nbl = pyma

Here is an example on how to sort and then access the results

In [6]:
#Sort by mean score ((forward+reverse)/2)
nbl_res.sort( 'mu_score' )
#Check top results
nbl_res.results.head()

,gene_name,forward_score,reverse_score,mu_score
0,FruMARCM-F000734_seg001,0.527808,0.593804,0.560806
1,DvGlutMARCM-F004348_seg001,0.591585,0.500109,0.545847
2,DvGlutMARCM-F002798_seg001,0.566495,0.510906,0.538700
3,FruMARCM-F001832_seg001,0.571805,0.499523,0.535664
4,DvGlutMARCM-F004142_seg001,0.545798,0.483806,0.514802


`nbl_res` also contains a wrapper that uses `pymaid.plot.plot3d` to generate results much like in R. Pymaid's `plot3d` can use two different backends: `vispy` (default) which opens a window much like R's `plot3d()`, and `plotly` that renders in the browser. Because it is fairly easy to embedd `plotly` in Jupyiter notebooks, we will use that for this example. Note that you have to run the code, otherwise the cell below will be empty!

In [7]:
import plotly.offline as pyoff

#We need to initialize plotly for interactive rendering with Jupyter notebooks
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [21]:
fig = nbl_res.plot3d( hits=2, backend='plotly', width = 1100)

Making neurons: 100%|██████████| 1/1 [00:00<00:00,  6.59it/s]
INFO  : Colormap:{'DvGlutMARCM-F004348_seg001': array([   0.,  255.,  255.]), '16': (0, 0, 0), 'FruMARCM-F000734_seg001': array([ 255.,    0.,    0.])} (pymaid.rmaid)
INFO  : Preparing neurons for plotting... (pymaid.plotting)
INFO  : Generating traces... (pymaid.plotting)
INFO  : Tracing done. (pymaid.plotting)
INFO  : Done. Plotted 15762 nodes and 0 connectors (pymaid.plotting)
INFO  : Use plotly.offline.plot(fig, filename="3d_plot.html") to plot. Optimised for Google Chrome. (pymaid.plotting)


In [22]:
pyoff.iplot(fig)

The red neuron is the CATMAID neuron we've blasted. The green and blue neurons are the top two hits.